In [1]:
import csv
import matplotlib.pyplot as plt

class F1Tracker:
    def __init__(self, experiment_name, save_csv=True, csv_path="f1_results.csv"):
        self.experiment_name = experiment_name
        self.percent_silenced = []
        self.f1_scores = []
        self.save_csv = save_csv
        self.csv_path = csv_path
        if self.save_csv:
            with open(self.csv_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["experiment", "percent_silenced", "f1_score"])

    def add(self, percent, f1_score):
        self.percent_silenced.append(percent)
        self.f1_scores.append(f1_score)
        if self.save_csv:
            with open(self.csv_path, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([self.experiment_name, percent, f1_score])

    def plot(self, show=True, save_path=None):
        plt.plot(self.percent_silenced, self.f1_scores, marker='o')
        plt.title(f"F1-score - {self.experiment_name}")
        plt.xlabel("% Neuronas Silenciadas")
        plt.ylabel("F1-score")
        plt.ylim(0, 1)
        plt.grid(True)
        if save_path:
            plt.savefig(save_path, bbox_inches='tight')
        if show:
            plt.show()
        plt.clf()

# Configs

In [2]:
import os
import torch
import json
import logging
import time
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import neurox.data.extraction.transformers_extractor as transformers_extractor
from neurox.data.writer import ActivationsWriter
import neurox.data.loader as data_loader
from transformers import AutoConfig
from tqdm import tqdm
import neurox.interpretation.linear_probe as linear_probe
import neurox.interpretation.utils as utils
import neurox.analysis.visualization as TransformersVisualizer
from sklearn.model_selection import train_test_split
from IPython.display import display
import neurox.interpretation.probeless as probeless
from neurox.interpretation.probeless import (
    get_neuron_ordering,
    get_neuron_ordering_for_all_tags
)
import ast
from torch.cuda.amp import autocast
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from matplotlib_venn import venn2
from sklearn.model_selection import train_test_split
from neurox.interpretation.linear_probe import get_top_neurons

In [3]:
import logging

# ==========================
# 📜 Configure Logging 
# ==========================

logger = logging.getLogger("synapse_logger")
logger.setLevel(logging.INFO)

# Avoid duplicates
if not logger.hasHandlers():

    # 📁 Handler 
    file_handler = logging.FileHandler("logs/synapse_extraction_csv_pth.log", mode="w")
    file_handler.setLevel(logging.INFO)

    # 🖥️ Handler 
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    # Format
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    # Add handlers to main logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

logger.info("🚀 Logging configured")


2025-07-02 12:54:25,385 - INFO - 🚀 Logging configured


In [4]:
# ==========================
# SYNAPSE Model Configuration
# ==========================

# 🧠 Select the model (options: "BERT", "BigBird", "DistilBERT", "Longformer")
MODEL = "BigBird"

# 📁 Paths based on model name
BASE_PATH = f"data/{MODEL}"
input_csv = f"{BASE_PATH}/{MODEL}_tokens_PT.csv"
output_csv = f"{BASE_PATH}/reduced/{MODEL}_tokens_reduced.csv"
labels_output_path = f"{BASE_PATH}/labels_numeric.txt"
label_mapping_path = f"{BASE_PATH}/labels_mapping.json"
activations_file = f"{BASE_PATH}/activations.json"
weights_path = f"{BASE_PATH}/best_model_{MODEL}.pth"

# 🔢 Number of labels
NUM_LABELS = 5


# 🔧 HuggingFace model mapping
MODEL_HF = {
    "BERT": "bert-base-uncased",
    "BigBird": "google/bigbird-roberta-base",
    "DistilBERT": "distilbert-base-uncased",
    "Longformer": "allenai/longformer-base-4096"
}[MODEL]

# ⚙️ Device selection
device = torch.device("cpu")

In [5]:
# ==========================
# Load Model and Weights
# ==========================
from transformers import AutoConfig

model = AutoModelForSequenceClassification.from_pretrained(MODEL_HF, num_labels=NUM_LABELS)

# Load trained weights from disk
state_dict = torch.load(weights_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

print(f"✅ Loaded {MODEL} with pretrained weights on {device}")

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded BigBird with pretrained weights on cpu


### Load dataset

In [6]:
reduction_ratio = 0.001

# ==========================
# ✅ Skip dataset reduction if already available
# ==========================
if os.path.exists(output_csv) and os.path.exists(labels_output_path):
    logger.info(f"⚡ Reduced dataset found: {output_csv}. Skipping reduction.")
    df_reduced = pd.read_csv(output_csv)
    with open(labels_output_path, "r") as f:
        labels = [int(line.strip()) for line in f]  # Labels as integers
    with open(label_mapping_path, "r") as f:
        label_mapping = json.load(f)  # Load label mapping
else:
    logger.info(f"🔄 Loading dataset from {input_csv}")

    chunk_size = 5000 
    total_rows = sum(1 for _ in open(input_csv)) - 1  # Total rows excluding header
    df_chunks = []

    logger.info(f"🔄 Processing {total_rows} rows in chunks of {chunk_size}...")

    with tqdm(total=total_rows, desc="Processing rows", unit=" rows") as pbar:
        for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
            # Convert `input_ids` from string to list of integers
            chunk['input_ids'] = chunk['input_ids'].apply(lambda x: list(map(int, x.strip("[]").split(","))))
            df_chunks.append(chunk)
            pbar.update(len(chunk))

    df = pd.concat(df_chunks, ignore_index=True)

    # ==========================
    # 🔢 Encode labels as integers
    # ==========================
    df['label'], unique_labels = pd.factorize(df["label"])
    label_mapping = {label: int(idx) for idx, label in enumerate(unique_labels)}

    # ==========================
    # 🧪 Reduce dataset maintaining class proportions
    # ==========================
    df_reduced, _ = train_test_split(df, train_size=reduction_ratio, stratify=df["label"], random_state=42)
    labels = df_reduced["label"].tolist()

    # ==========================
    # 💾 Save reduced dataset and labels
    # ==========================
    df_reduced.to_csv(output_csv, index=False)
    with open(labels_output_path, "w") as f:
        for label in labels:
            f.write(str(label) + "\n")

    with open(label_mapping_path, "w") as f:
        json.dump(label_mapping, f, indent=4)

    logger.info(f"✅ Reduced dataset saved to {output_csv}")
    logger.info(f"✅ Numeric labels saved to {labels_output_path}")
    logger.info(f"✅ Label mapping saved to {label_mapping_path}")


2025-07-02 12:54:26,985 - INFO - ⚡ Reduced dataset found: data/BigBird/reduced/BigBird_tokens_reduced.csv. Skipping reduction.


### Create Dataloader

In [7]:
# ==========================
# 📦 Create DataLoader
# ==========================
class SyscallDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.data.iloc[idx]['input_ids'])
        label = torch.tensor(self.data.iloc[idx]['label'])
        return input_ids, label

# Initialize DataLoader with reduced dataset
dataset = SyscallDataset(df_reduced)
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

logger.info("✅ Dataloader created")

# Ensure `input_ids` are lists of integers
if isinstance(df_reduced["input_ids"].iloc[0], str):
    df_reduced["input_ids"] = df_reduced["input_ids"].apply(lambda x: list(map(int, x.strip("[]").split(","))))


2025-07-02 12:54:26,997 - INFO - ✅ Dataloader created


# NeuroX

## Activation Extraction

In [8]:
if os.path.exists(activations_file):
    logger.info(f"⚡ Activations file found: {activations_file}. Skipping extraction.")
else:
    transformers_extractor.extract_representations(
        model, 
        df_reduced["input_ids"].tolist(),  # Pass preprocessed tokens directly
        activations_file,
        device=device,
    )

    logger.info(f"✅ Activations saved to {activations_file}")


2025-07-02 12:54:27,015 - INFO - ⚡ Activations file found: data/BigBird/activations.json. Skipping extraction.


## Load Activations

In [9]:
activations, num_layers = data_loader.load_activations(activations_file)
logger.info(f"✅ Loaded activations from {activations_file} with {num_layers} layers")

# Load sentence-level classification data using activations
tokens = data_loader.load_sentence_data(
    output_csv, labels_output_path, activations
)

# Create sentence-level tensors for classification
X, y, mapping = utils.create_tensors(
    tokens,
    activations,
    task_specific_tag="NN",
    task_type="classification"
)

label2idx, idx2label, src2idx, idx2src = mapping
logger.info("✅ Created input/output tensors and label mappings for classification")

2025-07-02 12:54:27,054 - INFO - ✅ Loaded activations from data/BigBird/activations.json with 12 layers
2025-07-02 12:54:27,063 - INFO - ✅ Created input/output tensors and label mappings for classification


Loading json activations from data/BigBird/activations.json...
50 12.0
Number of tokens:  50
length of source dictionary:  17
length of target dictionary:  5
50
Total instances: 50
['s']
Number of samples:  50
Stats: Labels with their frequencies in the final set
3 15
4 9
1 10
0 7
2 9


## Train linear probe

In [10]:
probe = linear_probe.train_logistic_regression_probe(X, y, lambda_l1=0.001, lambda_l2=0.001)
scores = linear_probe.evaluate_probe(probe, X, y, idx_to_class=idx2label)
logger.info(f"🎯 Probe evaluation results: {scores}")

top_neurons_probe, per_class_top_neurons = linear_probe.get_top_neurons(probe, percentage=0.1, class_to_idx=label2idx)
logger.info(f"🔍 Top global neurons: {top_neurons_probe}")
logger.info(f"🔍 Top neurons per class: {per_class_top_neurons}")

Clases en y_train: [0 1 2 3 4]
Training classification probe
Creating model...
Number of training instances: 50
Number of classes: 5


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0787


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0396


...

epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0140


Evaluating: 0it [00:00, ?it/s]

2025-07-02 12:54:27,182 - INFO - 🎯 Probe evaluation results: {'__OVERALL__': np.float64(0.96), '3': np.float64(1.0), '4': np.float64(1.0), '1': np.float64(1.0), '0': np.float64(1.0), '2': np.float64(0.7777777777777778)}
2025-07-02 12:54:27,187 - INFO - 🔍 Top global neurons: [8193 6145 6150 8200 8203 6155 6157 6158 8207 8206 4113 6162 6164 6165
 8214 6167 8220 6173 6178 8227 4131 8234 4139 8237 8240 6193 8241 4145
 8248 8255 6209 8257 8259 8260 6212 8266 8271 4181 8279 8280 8284 4191
 8287 8299 8302 8308 6262 8316 6272 8321 6274 8326 6280 8330 6283 8332
 8333 8335 8336 4240 8339 8341 8345 6303 6309 8357 8359 8360 8361 8363
 4268 6319 6141 8371 8374 4280 6329 6330 6331 8377 8381 6334 8382 8379
 8385 8386 6339 8388 8395 6347 8397 8401 8404 8413 6366 6367 8417 6372
 8425 8430 8431 8432 6382 6387 6389 6390 2295 6392 8437 6394 8445 8447
 8448 8451 6404 8455 8456 8457 6411 6413 6414 8463 8464 8461 8466 8465
 8468 6415 4379 8476 8475 2336 8482 8483 6434 8485 8486 8488 8489 8491
 8493 4397 4398

Score (accuracy) of the probe: 0.96


# Experiments

## Original performance

In [11]:
df = pd.read_csv(input_csv)

# 🎯 Select 50 random examples and reset index
sample_df = df.sample(n=50, random_state=42).reset_index(drop=True)

# 🧹 Convert "input_ids" and "attention_mask" from string to list format
def parse_list(x):
    return ast.literal_eval(x)

sample_df['input_ids'] = sample_df['input_ids'].apply(parse_list)
sample_df['attention_mask'] = sample_df['attention_mask'].apply(parse_list)

# 🔢 Encode labels to integers
label_encoder = LabelEncoder()
sample_df['label'] = label_encoder.fit_transform(sample_df['label'])
labels_list = sample_df['label'].tolist()

predictions_list = []
model.eval()
torch.cuda.empty_cache()

for i in range(len(sample_df)):
    input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids']).unsqueeze(0).to(device)
    attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask']).unsqueeze(0).to(device)

    with torch.no_grad():
        with autocast():
            device = torch.device("cpu")
            outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor)
            logits = outputs['logits']
            pred = torch.argmax(logits, dim=1).item()
            predictions_list.append(pred)

    del input_ids_tensor, attention_mask_tensor, outputs, logits
    torch.cuda.empty_cache()

# 📊 Compute evaluation metrics
accuracy = accuracy_score(labels_list, predictions_list)
f1 = f1_score(labels_list, predictions_list, average='weighted')
report_dict = classification_report(labels_list, predictions_list, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()

# Round for readability
report_df = report_df.round(4)

# Add accuracy as a separate row
accuracy_row = pd.DataFrame({'precision': accuracy, 'recall': accuracy, 'f1-score': accuracy, 'support': sum(report_df['support'])}, index=['accuracy'])
report_df = pd.concat([report_df, accuracy_row])

# 📝 Save or append to CSV

experiment_title = "🧪 Sample of 30 - Full Model Evaluation"
csv_report_path = f"{BASE_PATH}/classification_report_sample_eval.csv"

# Remove incorrect 'accuracy' row if it exists
report_df = report_df.drop("accuracy", errors="ignore")

# Append correct accuracy row
accuracy_row = pd.DataFrame({
    'precision': [""],
    'recall': [""],
    'f1-score': [accuracy],
    'support': [sum(report_df["support"])]
}, index=["overall_accuracy"])

# Combine
final_df = pd.concat([report_df, accuracy_row])

# Write to CSV with experiment title as a header
with open(csv_report_path, "a") as f:
    f.write(f"\n\n# {experiment_title}\n")
final_df.to_csv(csv_report_path, mode="a")

logger.info(f"📁 Appended classification report with title '{experiment_title}' to {csv_report_path}")


/var/folders/y4/zbxddgn52lx1dx5ktc7jch6m0000gn/T/ipykernel_41290/2165693082.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/Users/kikay/Documents/investigacion/.venv/lib/python3.9/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
2025-07-02 12:55:47,152 - INFO - 📁 Appended classification report with title '🧪 Sample of 30 - Full Model Evaluation' to data/BigBird/classification_report_sample_eval.csv


## Silence and evaluate neurons. Function definition

### Full silencing

In [12]:
def get_top_k_neurons_exact(probe, percentage: float) -> list[int]:
    """
    Return exactly N = round(total_neurons * percentage) neuron indices, sorted by importance.
    Importance is measured as the sum of absolute values of weights across all output classes.
    """
    weight_matrix = probe.linear.weight.detach().abs()  # [num_classes, num_neurons]
    importance = weight_matrix.sum(dim=0).cpu().numpy()  # [num_neurons]
    total_neurons = len(importance)
    top_n = round(total_neurons * percentage)

    sorted_indices = importance.argsort()[-top_n:]  # Top-N by importance
    return sorted_indices.tolist()

In [13]:
# ==========================
# Get encoder layers dynamically
# ==========================
def get_encoder_layers(model):
    if hasattr(model, "bert"):
        return model.bert.encoder.layer
    elif hasattr(model, "longformer"):
        return model.longformer.encoder.layer
    elif hasattr(model, "distilbert"):
        return model.distilbert.transformer.layer
    else:
        raise NotImplementedError("❌ Unsupported model architecture.")
def silence_top_global_percentage_and_evaluate(
    model,
    sample_df,
    labels_list,
    probe,
    label2idx,
    percentage=0.10,
    report_path=None,
    experiment_title=None,
    tracker=None  # nuevo argumento opcional
):
    hidden_dim = model.config.hidden_size
    num_layers = model.config.num_hidden_layers
    total_neurons = num_layers * hidden_dim

    top_neurons_global = get_top_k_neurons_exact(probe, percentage=percentage)
    logger.info(f"🔧 Silencing exactly {len(top_neurons_global)} neurons ({percentage:.2%} of total {total_neurons})")

    # Save neuron indices
    neurons_dir = f"{BASE_PATH}/neurons"
    os.makedirs(neurons_dir, exist_ok=True)
    json_path = f"{neurons_dir}/top_{int(percentage * 100)}p_neurons_global.json"
    with open(json_path, "w") as f:
        json.dump(top_neurons_global, f, indent=4)
    logger.info(f"📁 Saved neuron indices to {json_path}")

    # Register hooks per layer
    encoder_layers = get_encoder_layers(model)
    hook_handles = []
    for i in range(num_layers):
        indices_layer = [idx - i * hidden_dim for idx in top_neurons_global if i * hidden_dim <= idx < (i + 1) * hidden_dim]
        if indices_layer:
            logger.info(f"📌 Layer {i}: silencing {len(indices_layer)} neurons")
            handle = encoder_layers[i].output.register_forward_hook(make_cls_silence_hook(indices_layer))
            hook_handles.append(handle)

    # Inference
    model.eval()
    predictions = []
    for i in range(len(sample_df)):
        input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids']).unsqueeze(0).to(model.device)
        attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask']).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor)
            logits = outputs['logits']
            pred = torch.argmax(logits, dim=1).item()
            predictions.append(pred)

        del input_ids_tensor, attention_mask_tensor, outputs, logits
        torch.cuda.empty_cache()

    # Metrics
    accuracy = accuracy_score(labels_list, predictions)
    f1 = f1_score(labels_list, predictions, average='weighted')
    report_dict = classification_report(labels_list, predictions, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose().round(4)
    report_df = report_df.drop("accuracy", errors="ignore")

    accuracy_row = pd.DataFrame({
        'precision': [""],
        'recall': [""],
        'f1-score': [accuracy],
        'support': [sum(report_df["support"])]
    }, index=["overall_accuracy"])
    final_df = pd.concat([report_df, accuracy_row])

    # Save classification report
    if report_path is None:
        report_path = f"{BASE_PATH}/full_silencing.csv"
    os.makedirs(os.path.dirname(report_path), exist_ok=True)

    if experiment_title is None:
        experiment_title = f"Silencing top {percentage:.2%} global neurons"

    if not os.path.exists(report_path):
        with open(report_path, "w") as f:
            f.write(f"# {experiment_title}\n")
            final_df.to_csv(f)
    else:
        with open(report_path, "a") as f:
            f.write(f"\n\n# {experiment_title}\n")
        final_df.to_csv(report_path, mode="a")

    logger.info(f"🎯 Accuracy after silencing: {accuracy:.4f}")
    logger.info(f"📏 Weighted F1 Score: {f1:.4f}")
    logger.info(f"📋 Classification report saved to {report_path}")

    # Registrar F1 en el tracker si está definido
    if tracker:
        tracker.add(percent=percentage * 100, f1_score=f1)

    # Remove all hooks
    for handle in hook_handles:
        handle.remove()
    logger.info("✅ All hooks removed after evaluation")

In [14]:
def make_cls_silence_hook(indices):
    indices = [int(i) for i in indices]
    indices_tensor = torch.tensor(indices, dtype=torch.long) if indices else None

    def hook(module, input, output):
        if output.dim() == 3:
            new_output = output.clone()
            cls_token = new_output[:, 0, :]
            mask = torch.ones_like(cls_token)
            if indices_tensor is not None:
                local_indices = indices_tensor.to(new_output.device)
                mask[:, local_indices] = 0.0
            new_output[:, 0, :] = cls_token * mask
            return new_output
        return output
    return hook

In [16]:
# ==========================
# Get encoder layers dynamically
# ==========================
def get_encoder_layers(model):
    if hasattr(model, "bert"):
        return model.bert.encoder.layer
    elif hasattr(model, "longformer"):
        return model.longformer.encoder.layer
    elif hasattr(model, "distilbert"):
        return model.distilbert.transformer.layer
    else:
        raise NotImplementedError("❌ Unsupported model architecture.")

# ==========================
# Silence and evaluate global top-k neurons
# ==========================
def silence_top_global_percentage_and_evaluate(
    model,
    sample_df,
    labels_list,
    probe,
    label2idx,
    percentage=0.10,
    report_path=None,
    experiment_title=None
):
    hidden_dim = model.config.hidden_size
    num_layers = model.config.num_hidden_layers
    total_neurons = num_layers * hidden_dim

    top_neurons_global = get_top_k_neurons_exact(probe, percentage=percentage)
    logger.info(f"🔧 Silencing exactly {len(top_neurons_global)} neurons ({percentage:.2%} of total {total_neurons})")

    # Save neuron indices
    neurons_dir = f"{BASE_PATH}/neurons"
    os.makedirs(neurons_dir, exist_ok=True)
    json_path = f"{neurons_dir}/top_{int(percentage * 100)}p_neurons_global.json"
    with open(json_path, "w") as f:
        json.dump(top_neurons_global, f, indent=4)
    logger.info(f"📁 Saved neuron indices to {json_path}")

    # Register hooks per layer
    encoder_layers = get_encoder_layers(model)
    hook_handles = []
    for i in range(num_layers):
        indices_layer = [idx - i * hidden_dim for idx in top_neurons_global if i * hidden_dim <= idx < (i + 1) * hidden_dim]
        if indices_layer:
            logger.info(f"📌 Layer {i}: silencing {len(indices_layer)} neurons")
            handle = encoder_layers[i].output.register_forward_hook(make_cls_silence_hook(indices_layer))
            hook_handles.append(handle)

    # Inference
    model.eval()
    predictions = []
    for i in range(len(sample_df)):
        input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids']).unsqueeze(0).to(model.device)
        attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask']).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor)
            logits = outputs['logits']
            pred = torch.argmax(logits, dim=1).item()
            predictions.append(pred)

        del input_ids_tensor, attention_mask_tensor, outputs, logits
        torch.cuda.empty_cache()

    # Metrics
    accuracy = accuracy_score(labels_list, predictions)
    f1 = f1_score(labels_list, predictions, average='weighted')
    report_dict = classification_report(labels_list, predictions, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose().round(4)
    report_df = report_df.drop("accuracy", errors="ignore")

    accuracy_row = pd.DataFrame({
        'precision': [""],
        'recall': [""],
        'f1-score': [accuracy],
        'support': [sum(report_df["support"])]
    }, index=["overall_accuracy"])
    final_df = pd.concat([report_df, accuracy_row])

    # Save classification report
    if report_path is None:
        report_path = f"{BASE_PATH}/full_silencing.csv"
    os.makedirs(os.path.dirname(report_path), exist_ok=True)

    if experiment_title is None:
        experiment_title = f"Silencing top {percentage:.2%} global neurons"

    if not os.path.exists(report_path):
        with open(report_path, "w") as f:
            f.write(f"# {experiment_title}\n")
            final_df.to_csv(f)
    else:
        with open(report_path, "a") as f:
            f.write(f"\n\n# {experiment_title}\n")
        final_df.to_csv(report_path, mode="a")

    logger.info(f"🎯 Accuracy after silencing: {accuracy:.4f}")
    logger.info(f"📏 Weighted F1 Score: {f1:.4f}")
    logger.info(f"📋 Classification report saved to {report_path}")

    # Remove all hooks
    for handle in hook_handles:
        handle.remove()
    logger.info("✅ All hooks removed after evaluation")

## Global impact

In [21]:
percentages = [0.00, 0.025, 0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.65, 0.75, 0.8, 0.95]

for pct in percentages:
    silence_top_global_percentage_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        probe=probe,
        label2idx=label2idx,
        percentage=pct,
        experiment_title=f"Silencing {pct*100:.1f}% Global Neurons"
    )


2025-07-02 13:40:15,863 - INFO - 🔧 Silencing exactly 9216 neurons (0.00% of total 9216)
2025-07-02 13:40:15,870 - INFO - 📁 Saved neuron indices to data/BigBird/neurons/top_0p_neurons_global.json
2025-07-02 13:40:15,872 - INFO - 📌 Layer 0: silencing 768 neurons
2025-07-02 13:40:15,873 - INFO - 📌 Layer 1: silencing 768 neurons
2025-07-02 13:40:15,875 - INFO - 📌 Layer 2: silencing 768 neurons
2025-07-02 13:40:15,877 - INFO - 📌 Layer 3: silencing 768 neurons
2025-07-02 13:40:15,878 - INFO - 📌 Layer 4: silencing 768 neurons
2025-07-02 13:40:15,879 - INFO - 📌 Layer 5: silencing 768 neurons
2025-07-02 13:40:15,880 - INFO - 📌 Layer 6: silencing 768 neurons
2025-07-02 13:40:15,881 - INFO - 📌 Layer 7: silencing 768 neurons
2025-07-02 13:40:15,882 - INFO - 📌 Layer 8: silencing 768 neurons
2025-07-02 13:40:15,883 - INFO - 📌 Layer 9: silencing 768 neurons
2025-07-02 13:40:15,884 - INFO - 📌 Layer 10: silencing 768 neurons
2025-07-02 13:40:15,885 - INFO - 📌 Layer 11: silencing 768 neurons
/Users/kika

## Impact per class

In [22]:
def get_top_k_neurons_for_class_exact(probe, percentage: float, class_to_idx: dict, class_id: int) -> list[int]:
    """
    Return top-k neurons most important for a specific class, measured by absolute weight.
    """
    weight_matrix = probe.linear.weight.detach().abs()  # [num_classes, num_neurons]
    class_weights = weight_matrix[class_id]  # [num_neurons]
    total_neurons = class_weights.size(0)
    top_n = round(percentage * total_neurons)
    top_indices = class_weights.cpu().numpy().argsort()[-top_n:]
    return top_indices.tolist()

In [23]:
# ==========================
# Get encoder layers dynamically
# ==========================
def get_encoder_layers(model):
    if hasattr(model, "bert"):
        return model.bert.encoder.layer
    elif hasattr(model, "longformer"):
        return model.longformer.encoder.layer
    elif hasattr(model, "distilbert"):
        return model.distilbert.transformer.layer
    else:
        raise NotImplementedError("❌ Unsupported model architecture.")

# ==========================
# Silence and evaluate top per-class neurons
# ==========================

def silence_top_class_percentage_and_evaluate(
    model,
    sample_df,
    labels_list,
    probe,
    label2idx,
    class_id: int,
    percentage: float = 0.1,
    report_path: str = None,
    experiment_title: str = None
):
    class_name = f"class_{class_id}"
    hidden_dim = model.config.hidden_size
    num_layers = model.config.num_hidden_layers
    total_neurons = num_layers * hidden_dim

    # Get top neurons for specific class
    top_class_neurons = get_top_k_neurons_for_class_exact(
        probe, percentage=percentage, class_to_idx=label2idx, class_id=class_id
    )

    logger.info(f"🔧 Silencing {len(top_class_neurons)} neurons for class {class_id} ({percentage:.2%} of total)")

    # Save neurons to JSON
    neurons_dir = f"{BASE_PATH}/neurons"
    os.makedirs(neurons_dir, exist_ok=True)
    json_path = f"{neurons_dir}/top_{int(percentage * 100)}p_neurons_{class_name}.json"
    with open(json_path, "w") as f:
        json.dump(top_class_neurons, f, indent=4)
    logger.info(f"📁 Saved neuron indices to {json_path}")

    # Register hooks per layer
    encoder_layers = get_encoder_layers(model)
    hook_handles = []
    for i in range(num_layers):
        indices_layer = [idx - i * hidden_dim for idx in top_class_neurons if i * hidden_dim <= idx < (i + 1) * hidden_dim]
        if indices_layer:
            logger.info(f"📌 Layer {i}: silencing {len(indices_layer)} neurons for class {class_id}")
            handle = encoder_layers[i].output.register_forward_hook(make_cls_silence_hook(indices_layer))
            hook_handles.append(handle)

    # Inference
    model.eval()
    predictions = []
    for i in range(len(sample_df)):
        input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids']).unsqueeze(0).to(model.device)
        attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask']).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor)
            logits = outputs['logits']
            pred = torch.argmax(logits, dim=1).item()
            predictions.append(pred)

        del input_ids_tensor, attention_mask_tensor, outputs, logits
        torch.cuda.empty_cache()

    # Metrics
    accuracy = accuracy_score(labels_list, predictions)
    f1 = f1_score(labels_list, predictions, average='weighted')
    report_dict = classification_report(labels_list, predictions, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose().round(4)
    report_df = report_df.drop("accuracy", errors="ignore")

    accuracy_row = pd.DataFrame({
        'precision': [""],
        'recall': [""],
        'f1-score': [accuracy],
        'support': [sum(report_df["support"])]
    }, index=["overall_accuracy"])
    final_df = pd.concat([report_df, accuracy_row])

    # Save classification report
    if report_path is None:
        report_path = f"{BASE_PATH}/class_silencing.csv"
    os.makedirs(os.path.dirname(report_path), exist_ok=True)

    if experiment_title is None:
        experiment_title = f"Silencing top {percentage:.2%} neurons for class {class_id}"

    if not os.path.exists(report_path):
        with open(report_path, "w") as f:
            f.write(f"# {experiment_title}\n")
            final_df.to_csv(f)
    else:
        with open(report_path, "a") as f:
            f.write(f"\n\n# {experiment_title}\n")
        final_df.to_csv(report_path, mode="a")

    logger.info(f"🎯 Accuracy after class-specific silencing: {accuracy:.4f}")
    logger.info(f"📏 Weighted F1 Score: {f1:.4f}")
    logger.info(f"📋 Classification report saved to {report_path}")

    # Remove all hooks
    for handle in hook_handles:
        handle.remove()
    logger.info("✅ All hooks removed after evaluation")

In [25]:
percentages = [0.00, 0.025, 0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.65, 0.75, 0.8, 0.95]

target_class_id = 4

for pct in percentages:
    silence_top_class_percentage_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        probe=probe,
        label2idx=label2idx,
        class_id=target_class_id,
        percentage=pct,
        experiment_title=f"Silencing {pct*100:.1f}% of Neurons for Class {target_class_id}"
    )


2025-07-02 14:05:57,158 - INFO - 🔧 Silencing 9216 neurons for class 4 (0.00% of total)
2025-07-02 14:05:57,165 - INFO - 📁 Saved neuron indices to data/BigBird/neurons/top_0p_neurons_class_4.json
2025-07-02 14:05:57,167 - INFO - 📌 Layer 0: silencing 768 neurons for class 4
2025-07-02 14:05:57,169 - INFO - 📌 Layer 1: silencing 768 neurons for class 4
2025-07-02 14:05:57,171 - INFO - 📌 Layer 2: silencing 768 neurons for class 4
2025-07-02 14:05:57,173 - INFO - 📌 Layer 3: silencing 768 neurons for class 4
2025-07-02 14:05:57,175 - INFO - 📌 Layer 4: silencing 768 neurons for class 4
2025-07-02 14:05:57,176 - INFO - 📌 Layer 5: silencing 768 neurons for class 4
2025-07-02 14:05:57,178 - INFO - 📌 Layer 6: silencing 768 neurons for class 4
2025-07-02 14:05:57,179 - INFO - 📌 Layer 7: silencing 768 neurons for class 4
2025-07-02 14:05:57,180 - INFO - 📌 Layer 8: silencing 768 neurons for class 4
2025-07-02 14:05:57,181 - INFO - 📌 Layer 9: silencing 768 neurons for class 4
2025-07-02 14:05:57,182 -

In [26]:
percentages = [0.00, 0.025, 0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.65, 0.75, 0.8, 0.95]

target_class_id = 3

for pct in percentages:
    silence_top_class_percentage_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        probe=probe,
        label2idx=label2idx,
        class_id=target_class_id,
        percentage=pct,
        experiment_title=f"Silencing {pct*100:.1f}% of Neurons for Class {target_class_id}"
    )


2025-07-02 14:29:07,743 - INFO - 🔧 Silencing 9216 neurons for class 3 (0.00% of total)
2025-07-02 14:29:07,745 - INFO - 📁 Saved neuron indices to data/BigBird/neurons/top_0p_neurons_class_3.json
2025-07-02 14:29:07,746 - INFO - 📌 Layer 0: silencing 768 neurons for class 3
2025-07-02 14:29:07,746 - INFO - 📌 Layer 1: silencing 768 neurons for class 3
2025-07-02 14:29:07,747 - INFO - 📌 Layer 2: silencing 768 neurons for class 3
2025-07-02 14:29:07,748 - INFO - 📌 Layer 3: silencing 768 neurons for class 3
2025-07-02 14:29:07,748 - INFO - 📌 Layer 4: silencing 768 neurons for class 3
2025-07-02 14:29:07,749 - INFO - 📌 Layer 5: silencing 768 neurons for class 3
2025-07-02 14:29:07,750 - INFO - 📌 Layer 6: silencing 768 neurons for class 3
2025-07-02 14:29:07,750 - INFO - 📌 Layer 7: silencing 768 neurons for class 3
2025-07-02 14:29:07,751 - INFO - 📌 Layer 8: silencing 768 neurons for class 3
2025-07-02 14:29:07,751 - INFO - 📌 Layer 9: silencing 768 neurons for class 3
2025-07-02 14:29:07,752 -

In [27]:
percentages = [0.00, 0.025, 0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.65, 0.75, 0.8, 0.95]

target_class_id = 2
for pct in percentages:
    silence_top_class_percentage_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        probe=probe,
        label2idx=label2idx,
        class_id=target_class_id,
        percentage=pct,
        experiment_title=f"Silencing {pct*100:.1f}% of Neurons for Class {target_class_id}"
    )


2025-07-02 14:52:15,462 - INFO - 🔧 Silencing 9216 neurons for class 2 (0.00% of total)
2025-07-02 14:52:15,464 - INFO - 📁 Saved neuron indices to data/BigBird/neurons/top_0p_neurons_class_2.json
2025-07-02 14:52:15,464 - INFO - 📌 Layer 0: silencing 768 neurons for class 2
2025-07-02 14:52:15,465 - INFO - 📌 Layer 1: silencing 768 neurons for class 2
2025-07-02 14:52:15,466 - INFO - 📌 Layer 2: silencing 768 neurons for class 2
2025-07-02 14:52:15,467 - INFO - 📌 Layer 3: silencing 768 neurons for class 2
2025-07-02 14:52:15,467 - INFO - 📌 Layer 4: silencing 768 neurons for class 2
2025-07-02 14:52:15,468 - INFO - 📌 Layer 5: silencing 768 neurons for class 2
2025-07-02 14:52:15,468 - INFO - 📌 Layer 6: silencing 768 neurons for class 2
2025-07-02 14:52:15,469 - INFO - 📌 Layer 7: silencing 768 neurons for class 2
2025-07-02 14:52:15,470 - INFO - 📌 Layer 8: silencing 768 neurons for class 2
2025-07-02 14:52:15,470 - INFO - 📌 Layer 9: silencing 768 neurons for class 2
2025-07-02 14:52:15,471 -

In [28]:
percentages = [0.00, 0.025, 0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.65, 0.75, 0.8, 0.95]

target_class_id = 1

for pct in percentages:
    silence_top_class_percentage_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        probe=probe,
        label2idx=label2idx,
        class_id=target_class_id,
        percentage=pct,
        experiment_title=f"Silencing {pct*100:.1f}% of Neurons for Class {target_class_id}"
    )


2025-07-02 15:16:38,437 - INFO - 🔧 Silencing 9216 neurons for class 1 (0.00% of total)
2025-07-02 15:16:38,441 - INFO - 📁 Saved neuron indices to data/BigBird/neurons/top_0p_neurons_class_1.json
2025-07-02 15:16:38,442 - INFO - 📌 Layer 0: silencing 768 neurons for class 1
2025-07-02 15:16:38,442 - INFO - 📌 Layer 1: silencing 768 neurons for class 1
2025-07-02 15:16:38,443 - INFO - 📌 Layer 2: silencing 768 neurons for class 1
2025-07-02 15:16:38,444 - INFO - 📌 Layer 3: silencing 768 neurons for class 1
2025-07-02 15:16:38,445 - INFO - 📌 Layer 4: silencing 768 neurons for class 1
2025-07-02 15:16:38,446 - INFO - 📌 Layer 5: silencing 768 neurons for class 1
2025-07-02 15:16:38,447 - INFO - 📌 Layer 6: silencing 768 neurons for class 1
2025-07-02 15:16:38,448 - INFO - 📌 Layer 7: silencing 768 neurons for class 1
2025-07-02 15:16:38,449 - INFO - 📌 Layer 8: silencing 768 neurons for class 1
2025-07-02 15:16:38,449 - INFO - 📌 Layer 9: silencing 768 neurons for class 1
2025-07-02 15:16:38,450 -

In [29]:
percentages = [0.00, 0.025, 0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.65, 0.75, 0.8, 0.95]

target_class_id = 0

for pct in percentages:
    silence_top_class_percentage_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        probe=probe,
        label2idx=label2idx,
        class_id=target_class_id,
        percentage=pct,
        experiment_title=f"Silencing {pct*100:.1f}% of Neurons for Class {target_class_id}"
    )


2025-07-03 11:46:11,749 - INFO - 🔧 Silencing 9216 neurons for class 0 (0.00% of total)
2025-07-03 11:46:11,751 - INFO - 📁 Saved neuron indices to data/BigBird/neurons/top_0p_neurons_class_0.json
2025-07-03 11:46:11,752 - INFO - 📌 Layer 0: silencing 768 neurons for class 0
2025-07-03 11:46:11,753 - INFO - 📌 Layer 1: silencing 768 neurons for class 0
2025-07-03 11:46:11,754 - INFO - 📌 Layer 2: silencing 768 neurons for class 0
2025-07-03 11:46:11,754 - INFO - 📌 Layer 3: silencing 768 neurons for class 0
2025-07-03 11:46:11,755 - INFO - 📌 Layer 4: silencing 768 neurons for class 0
2025-07-03 11:46:11,755 - INFO - 📌 Layer 5: silencing 768 neurons for class 0
2025-07-03 11:46:11,756 - INFO - 📌 Layer 6: silencing 768 neurons for class 0
2025-07-03 11:46:11,757 - INFO - 📌 Layer 7: silencing 768 neurons for class 0
2025-07-03 11:46:11,758 - INFO - 📌 Layer 8: silencing 768 neurons for class 0
2025-07-03 11:46:11,758 - INFO - 📌 Layer 9: silencing 768 neurons for class 0
2025-07-03 11:46:11,759 -

# Attacks

## FGSM

In [ ]:
from torch.nn import CrossEntropyLoss

def run_fgsm_attack_and_evaluate(
    model,
    sample_df,
    labels_list,
    epsilon: float = 0.1,
    report_path: str = f"{BASE_PATH}/fgsm.csv",
    experiment_title: str = None
):
    logger.info(f"⚔️ Running FGSM attack with ε = {epsilon}")
    model.eval()
    predictions_fgsm = []
    loss_fn = CrossEntropyLoss()

    is_longformer = hasattr(model, "longformer")

    for i in range(len(sample_df)):
        # ===============================
        # 🔢 Prepare input tensors
        # ===============================
        input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids'], dtype=torch.long).unsqueeze(0).to(model.device)
        attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask'], dtype=torch.long).unsqueeze(0).to(model.device)
        true_label = torch.tensor([labels_list[i]], dtype=torch.long).to(model.device)

        # ===============================
        # 🔍 Extract embeddings as leaf tensor
        # ===============================
        with torch.no_grad():
            embedding_output = model.base_model.embeddings(input_ids_tensor)
        embeds = embedding_output.clone().detach().requires_grad_(True)

        # ===============================
        # 🔁 Forward pass
        # ===============================
        if is_longformer:
            global_attention_mask = torch.zeros_like(attention_mask_tensor)
            global_attention_mask[:, 0] = 1
            outputs = model(
                inputs_embeds=embeds,
                attention_mask=attention_mask_tensor,
                global_attention_mask=global_attention_mask
            )
        else:
            outputs = model(
                inputs_embeds=embeds,
                attention_mask=attention_mask_tensor
            )

        logits = outputs.logits
        loss = loss_fn(logits, true_label)

        # ===============================
        # 🔁 Backward pass
        # ===============================
        model.zero_grad()
        loss.backward()

        # ===============================
        # ⚔️ FGSM perturbation
        # ===============================
        perturbation = epsilon * embeds.grad.data.sign()
        adv_embeds = embeds + perturbation

        # ===============================
        # 🔮 Inference with adversarial input
        # ===============================
        with torch.no_grad():
            if is_longformer:
                adv_outputs = model(
                    inputs_embeds=adv_embeds,
                    attention_mask=attention_mask_tensor,
                    global_attention_mask=global_attention_mask
                )
            else:
                adv_outputs = model(
                    inputs_embeds=adv_embeds,
                    attention_mask=attention_mask_tensor
                )
            adv_logits = adv_outputs.logits
            pred = torch.argmax(adv_logits, dim=1).item()
            predictions_fgsm.append(pred)

        del input_ids_tensor, attention_mask_tensor, embeds, adv_embeds, outputs, adv_outputs, logits, adv_logits
        torch.cuda.empty_cache()

    # ===============================
    # 📊 Evaluation
    # ===============================
    accuracy = accuracy_score(labels_list, predictions_fgsm)
    f1 = f1_score(labels_list, predictions_fgsm, average='weighted')
    report_dict = classification_report(labels_list, predictions_fgsm, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose().round(4).drop("accuracy", errors="ignore")

    accuracy_row = pd.DataFrame({
        'precision': [""],
        'recall': [""],
        'f1-score': [accuracy],
        'support': [sum(report_df["support"])]
    }, index=["overall_accuracy"])
    final_df = pd.concat([report_df, accuracy_row])

    # ===============================
    # 💾 Save report
    # ===============================
    if experiment_title is None:
        experiment_title = f"FGSM Attack (ε = {epsilon})"
    if not os.path.exists(report_path):
        with open(report_path, "w") as f:
            f.write(f"# {experiment_title}\n")
            final_df.to_csv(f)
    else:
        with open(report_path, "a") as f:
            f.write(f"\n\n# {experiment_title}\n")
        final_df.to_csv(report_path, mode="a")

    logger.info(f"🎯 Accuracy under FGSM (ε={epsilon}): {accuracy:.4f}")
    logger.info(f"📏 Weighted F1 Score: {f1:.4f}")
    logger.info(f"📋 Classification report saved to {report_path}")

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
model.to("cpu")
device = torch.device("cpu")


In [ ]:
run_fgsm_attack_and_evaluate(
    model=model,
    sample_df=sample_df,
    labels_list=labels_list,
    epsilon=0.1,
    experiment_title="FGSM Adversarial Attack with ε = 0.1"
)

## Random Noise

In [ ]:
def run_random_noise_attack(
    model,
    sample_df,
    labels_list,
    epsilon=0.3,
    device="cpu"
):
    import torch
    from sklearn.metrics import accuracy_score, f1_score, classification_report

    logger.info(f"🎲 Running random noise attack (epsilon={epsilon})")
    model.to(device)
    model.eval()
    predictions_noise = []

    is_longformer = hasattr(model, "longformer")

    for i in range(len(sample_df)):
        input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids'], dtype=torch.long).unsqueeze(0).to(device)
        attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask'], dtype=torch.long).unsqueeze(0).to(device)

        with torch.no_grad():
            embeddings = model.base_model.embeddings(input_ids_tensor)
            noisy_embeddings = embeddings + epsilon * torch.randn_like(embeddings)

            if is_longformer:
                global_attention_mask = torch.zeros_like(attention_mask_tensor)
                global_attention_mask[:, 0] = 1
                outputs = model(
                    inputs_embeds=noisy_embeddings,
                    attention_mask=attention_mask_tensor,
                    global_attention_mask=global_attention_mask
                )
            else:
                outputs = model(
                    inputs_embeds=noisy_embeddings,
                    attention_mask=attention_mask_tensor
                )

            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()
            predictions_noise.append(pred)

    accuracy = accuracy_score(labels_list, predictions_noise)
    f1 = f1_score(labels_list, predictions_noise, average='weighted')
    report = classification_report(labels_list, predictions_noise)

    logger.info(f"📉 Random noise attack results (epsilon={epsilon}):")
    logger.info(f"Accuracy: {accuracy:.4f} | F1 Score: {f1:.4f}")
    logger.info(f"\nClassification Report:\n{report}")

    return accuracy, f1, predictions_noise

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report

epsilons = np.linspace(0.1, 1.0, 10)  # 10 valores de 0.1 a 1.0
results = []

for epsilon in epsilons:
    logger.info(f"\n🎲 Running random noise attack with ε={epsilon:.2f}")

    acc_noise, f1_noise, preds_noise = run_random_noise_attack(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        epsilon=epsilon,
        device="cpu"
    )

    logger.info(f"📉 Results for ε={epsilon:.2f} → Accuracy: {acc_noise:.4f} | F1 Score: {f1_noise:.4f}")

    results.append({
        "epsilon": round(epsilon, 2),
        "accuracy": acc_noise,
        "f1_score": f1_noise
    })

# Convertimos resultados a dataframe
noise_sweep_df = pd.DataFrame(results)
noise_sweep_df.to_csv("results/random_noise_sweep_up_to_1.csv", index=False)

logger.info("📁 Random noise sweep results saved to 'results/random_noise_sweep_up_to_1.csv'")


# Conjuntos disjuntos de neuronas

In [30]:
import glob

neurons_dir = f"{BASE_PATH}/neurons"
json_files = sorted(glob.glob(f"{neurons_dir}/top_*p_neurons_class_*.json"))
total_neurons = 9216
percentiles = [0.10, 0.25, 0.50]

for cutoff in percentiles:
    rankings = {}
    for path in json_files:
        class_id = int(path.split("_")[-1].split(".")[0])
        with open(path, "r") as f:
            full_list = json.load(f)
            top_k = full_list[:int(cutoff * total_neurons)]
            rankings[class_id] = set(top_k)

    exclusive_neurons = {}
    for i in range(5):
        others_union = set().union(*[rankings[j] for j in range(5) if j != i])
        exclusive = rankings[i] - others_union
        exclusive_neurons[i] = sorted(list(exclusive))

    exclusive_dir = f"{BASE_PATH}/exclusive_neurons"
    os.makedirs(exclusive_dir, exist_ok=True)

    for class_id, neurons in exclusive_neurons.items():
        path = f"{exclusive_dir}/exclusive_top{int(cutoff*100)}_class_{class_id}.json"
        with open(path, "w") as f:
            json.dump(neurons, f, indent=4)
        logger.info(f"✅ Saved exclusive neurons for class {class_id} ({cutoff*100:.0f}%) → {path}")

2025-07-03 12:10:41,047 - INFO - ✅ Saved exclusive neurons for class 0 (10%) → data/BigBird/exclusive_neurons/exclusive_top10_class_0.json
2025-07-03 12:10:41,048 - INFO - ✅ Saved exclusive neurons for class 1 (10%) → data/BigBird/exclusive_neurons/exclusive_top10_class_1.json
2025-07-03 12:10:41,049 - INFO - ✅ Saved exclusive neurons for class 2 (10%) → data/BigBird/exclusive_neurons/exclusive_top10_class_2.json
2025-07-03 12:10:41,049 - INFO - ✅ Saved exclusive neurons for class 3 (10%) → data/BigBird/exclusive_neurons/exclusive_top10_class_3.json
2025-07-03 12:10:41,050 - INFO - ✅ Saved exclusive neurons for class 4 (10%) → data/BigBird/exclusive_neurons/exclusive_top10_class_4.json
2025-07-03 12:10:41,070 - INFO - ✅ Saved exclusive neurons for class 0 (25%) → data/BigBird/exclusive_neurons/exclusive_top25_class_0.json
2025-07-03 12:10:41,071 - INFO - ✅ Saved exclusive neurons for class 1 (25%) → data/BigBird/exclusive_neurons/exclusive_top25_class_1.json
2025-07-03 12:10:41,072 - I

In [31]:
def silence_exclusive_neurons_and_evaluate(
    model,
    sample_df,
    labels_list,
    class_id: int,
    exclusive_json_path: str,
    report_path: str = None,
    experiment_title: str = None
):
    # Leer neuronas exclusivas
    with open(exclusive_json_path, "r") as f:
        exclusive_neurons = json.load(f)

    logger.info(f"🔧 Silencing {len(exclusive_neurons)} exclusive neurons for class {class_id}")

    hidden_dim = model.config.hidden_size
    num_layers = model.config.num_hidden_layers

    # Registrar hooks para cada capa con las neuronas exclusivas que correspondan
    encoder_layers = get_encoder_layers(model)
    hook_handles = []
    for i in range(num_layers):
        indices_layer = [idx - i * hidden_dim for idx in exclusive_neurons if i * hidden_dim <= idx < (i + 1) * hidden_dim]
        if indices_layer:
            logger.info(f"📌 Layer {i}: silencing {len(indices_layer)} exclusive neurons")
            handle = encoder_layers[i].output.register_forward_hook(make_cls_silence_hook(indices_layer))
            hook_handles.append(handle)

    # Inference
    model.eval()
    predictions = []
    for i in range(len(sample_df)):
        input_ids_tensor = torch.tensor(sample_df.loc[i, 'input_ids']).unsqueeze(0).to(model.device)
        attention_mask_tensor = torch.tensor(sample_df.loc[i, 'attention_mask']).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor)
            logits = outputs['logits']
            pred = torch.argmax(logits, dim=1).item()
            predictions.append(pred)

        del input_ids_tensor, attention_mask_tensor, outputs, logits
        torch.cuda.empty_cache()

    # Métricas
    accuracy = accuracy_score(labels_list, predictions)
    f1 = f1_score(labels_list, predictions, average='weighted')
    report_dict = classification_report(labels_list, predictions, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose().round(4)
    report_df = report_df.drop("accuracy", errors="ignore")

    accuracy_row = pd.DataFrame({
        'precision': [""],
        'recall': [""],
        'f1-score': [accuracy],
        'support': [sum(report_df["support"])]
    }, index=["overall_accuracy"])
    final_df = pd.concat([report_df, accuracy_row])

    if report_path is None:
        report_path = f"{BASE_PATH}/exclusive_silencing.csv"
    os.makedirs(os.path.dirname(report_path), exist_ok=True)

    if experiment_title is None:
        experiment_title = f"Silencing exclusive neurons for class {class_id}"

    if not os.path.exists(report_path):
        with open(report_path, "w") as f:
            f.write(f"# {experiment_title}\n")
            final_df.to_csv(f)
    else:
        with open(report_path, "a") as f:
            f.write(f"\n\n# {experiment_title}\n")
        final_df.to_csv(report_path, mode="a")

    logger.info(f"🎯 Accuracy after exclusive silencing: {accuracy:.4f}")
    logger.info(f"📏 Weighted F1 Score: {f1:.4f}")
    logger.info(f"📋 Classification report saved to {report_path}")

    # Quitar hooks
    for handle in hook_handles:
        handle.remove()
    logger.info("✅ All hooks removed after evaluation")

    return f1

In [32]:
for class_id in range(5):
    path = f"{BASE_PATH}/exclusive_neurons/exclusive_top4608_class_{class_id}.json"
    silence_exclusive_neurons_and_evaluate(
        model=model,
        sample_df=sample_df,
        labels_list=labels_list,
        class_id=class_id,
        exclusive_json_path=path,
        report_path=f"{BASE_PATH}/exclusive_silencing.csv",
        experiment_title=f"Silencing exclusive neurons for class {class_id}"
    )

FileNotFoundError: [Errno 2] No such file or directory: 'data/BigBird/exclusive_neurons/exclusive_top4608_class_0.json'

In [ ]:
for cutoff in [10, 25, 50]:
    for class_id in range(5):
        path = f"{BASE_PATH}/exclusive_neurons/exclusive_top{cutoff}_class_{class_id}.json"
        silence_exclusive_neurons_and_evaluate(
            model=model,
            sample_df=sample_df,
            labels_list=labels_list,
            class_id=class_id,
            exclusive_json_path=path,
            report_path=f"{BASE_PATH}/exclusive_silencing_{cutoff}p.csv",
            experiment_title=f"Silencing exclusive neurons ({cutoff}%) for class {class_id}"
        )